In [ ]:
from langchain.chains import RetrievalQA  # 检索QA链，在文档上进行检索
# from langchain.chat_models import ChatOpenAI  # openai模型
# from langchain_community.chat_models import ChatOllama
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings  # ollama嵌入
from langchain.document_loaders import CSVLoader  # 文档加载器，采用csv格式存储
# 导入向量存储索引创建器
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch  # 向量存储

from IPython.display import display, Markdown  # 在jupyter显示信息的工具
import pandas as pd

file = "../data/OutdoorClothingCatalog_1000.csv"

# 使用langchain文档加载器对数据进行导入
loader = CSVLoader(file_path=file)

# 使用pandas导入数据，用以查看
data = pd.read_csv(file, usecols=[1, 2])
data.head()

,name,description
0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
1,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
2,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
3,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
4,EcoFlex 3L Storm Pants,Our new TEK O2 technology makes our four-seaso...


In [19]:
llm = ChatOllama(model="qwen2.5", temperature=0.0)  # 使用ollama模型

In [25]:
# 创建 Ollama 嵌入模型实例
embeddings = OllamaEmbeddings(model="qwen2.5")  # 或其他你在 Ollama 中安装的模型名称

try:
    # Create the document loader
    file = "../data/OutdoorClothingCatalog_1000.csv"
    loader = CSVLoader(file_path=file)

    # Create vector store index
    index = VectorstoreIndexCreator(
        vectorstore_cls=DocArrayInMemorySearch,
        embedding=embeddings,  # 添加 embedding 参数
    ).from_loaders([loader])

except Exception as e:
    print(f"Error creating index: {str(e)}")

/opt/anaconda3/envs/llm_env/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [36]:
# query = "请用markdown表格的方式列出所有具有防晒功能的衬衫，对每件衬衫描述进行总结"


In [38]:
# 创建一个文档加载器，通过csv格式加载
file = "../data/OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file_path=file)
docs = loader.load()

# 查看单个文档，每个文档对应于CSV中的一行数据
docs[0]

Document(metadata={'source': '../data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [41]:
embed = embeddings.embed_query("你好呀，我的名字叫小可爱")

In [43]:
print(embed)

[-0.003120341, 0.0022507957, -0.0013351035, 0.007695697, 0.0015686533, 0.005230901, 0.017338198, 0.009874855, 0.0082209725, 0.010216273, 0.007303077, -0.00782827, -0.0049148505, 0.0065068407, 0.015927149, 0.0006006338, -0.006083167, -0.0013691054, 0.014594379, -0.00027456367, -0.078011155, 0.0017279616, -0.0058842027, 0.0088709295, -0.0077999313, -0.013825365, 0.00378206, 0.007287196, -0.0012077177, 0.005058798, -0.0009885576, -0.008041202, 0.012330638, 0.0008490452, -0.0104605025, -3.3164226e-05, 0.013085187, 0.022971028, 0.0054860674, -0.024419038, -0.005412033, -0.009300795, -0.0051468913, 0.0117354775, -0.0026719887, 0.005844883, 0.0042454726, -0.015252781, 0.00043747667, 0.0010378505, 0.0010678805, 0.00089566055, -0.0045838645, -0.008243217, 0.004113302, -0.00048500634, -0.00082321826, 0.008856631, 0.0039684228, -0.007058471, 0.012397822, 0.009775793, 0.0076929354, -0.00848801, -0.005319518, 0.0072641624, -0.0059692105, 0.018278433, -0.015354556, 0.015472404, 0.0019011557, 0.01129